In [20]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

In [21]:
import xgboost as xgb
import scipy

In [22]:
from sklearn import neural_network

In [67]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [68]:
train_copy = train
train_copy = train_copy.replace(-1, np.NaN)
train['missing_volume'] = train_copy.isnull().sum(axis = 1)

In [69]:
test_copy = test
test_copy = test_copy.replace(-1, np.NaN)
test['missing_volume'] = test_copy.isnull().sum(axis = 1)

In [55]:
cat_features = [a for a in train.columns if a.endswith('cat')]

In [70]:
y = train['target']
X = train.drop('target',axis = 1)

In [13]:

import sklearn
import time

In [ ]:
bin_col = [col for col in train.columns if '_bin' in col]
bin_col_ind = [col for col in bin_col if '_ind' in col]
train_bin_ind_x = train[bin_col_ind]
train['binary_ind_sum'] = train_bin_ind_x.sum(axis = 1)

In [7]:
bin_col = [col for col in test.columns if '_bin' in col]
bin_col_ind = [col for col in bin_col if '_ind' in col]
test_bin_ind_x = test[bin_col_ind]
test['binary_ind_sum'] = test_bin_ind_x.sum(axis = 1)

In [46]:
bin_col = [col for col in train.columns if '_bin' in col]
bin_col_ind = [col for col in bin_col if '_ind' in col]
train_bin_ind_x = train[bin_col_ind]
ref_row = train_bin_ind_x.iloc[0,:]
bin_ind_diff = train_bin_ind_x - ref_row
train['ind_bin_diff_sum'] = bin_ind_diff.abs().sum(axis = 1)

In [47]:
bin_col = [col for col in test.columns if '_bin' in col]
bin_col_ind = [col for col in bin_col if '_ind' in col]
test_bin_ind_x = test[bin_col_ind]
ref_row = test_bin_ind_x.iloc[0,:]
bin_ind_diff = test_bin_ind_x - ref_row
test['ind_bin_diff_sum'] = bin_ind_diff.abs().sum(axis = 1)

In [71]:
# Preprocessing 
id_test = test['id']
test = test.drop(['id'], axis = 1)
X = X.drop('id',axis = 1)

In [12]:


col_to_drop = X.columns[X.columns.str.startswith('ps_calc_')]
X = X.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  



In [72]:
col_to_drop2 = ['ps_calc_01', 'ps_calc_04','ps_calc_02', 'ps_calc_03', 'ps_calc_05',
       'ps_calc_06',  'ps_calc_08', 'ps_calc_09', 'ps_calc_10',
       'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_ind_10_bin','ps_ind_11_bin','ps_ind_13_bin','ps_car_10_cat']
X = X.drop(col_to_drop2, axis=1)  
test = test.drop(col_to_drop2, axis=1)  

In [74]:
cat_features = [a for a in X.columns if a.endswith('cat')]

In [75]:
cat_features

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_11_cat']

In [76]:


for column in cat_features:
    temp = pd.get_dummies(pd.Series(X[column]))
    temp.rename(columns=lambda x: str(x)+column, inplace=True)
    X = pd.concat([X,temp],axis=1)
    X = X.drop([column],axis=1)
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(test[column]))
    temp.rename(columns=lambda x: str(x)+column, inplace=True)
    test = pd.concat([test,temp],axis=1)
    test = test.drop([column],axis=1)


In [32]:
X.columns

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin',
       ...
       '95ps_car_11_cat', '96ps_car_11_cat', '97ps_car_11_cat',
       '98ps_car_11_cat', '99ps_car_11_cat', '100ps_car_11_cat',
       '101ps_car_11_cat', '102ps_car_11_cat', '103ps_car_11_cat',
       '104ps_car_11_cat'],
      dtype='object', length=209)

In [76]:
test.columns

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin',
       ...
       '95ps_car_11_cat', '96ps_car_11_cat', '97ps_car_11_cat',
       '98ps_car_11_cat', '99ps_car_11_cat', '100ps_car_11_cat',
       '101ps_car_11_cat', '102ps_car_11_cat', '103ps_car_11_cat',
       '104ps_car_11_cat'],
      dtype='object', length=208)

In [77]:
X.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [78]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        #X = np.array(X)
        #y = np.array(y)
        #T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X.iloc[train_idx]
                y_train = y.iloc[train_idx]
                X_holdout = X.iloc[test_idx]
#                y_holdout = y[test_idx]
                pos = pd.Series(y_train == 1)
                X_train = pd.concat([X_train, X_train.loc[pos]], axis=0)
                y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
    
                idx = np.arange(len(X_train))
                np.random.shuffle(idx)
                X_train = X_train.iloc[idx]
                y_train = y_train.iloc[idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = scipy.stats.hmean(S_test_i, axis=1)
            #S_test[:, i] = np.median(S_test_i, axis = 1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res


In [79]:
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['seed'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['seed'] = 99


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['seed'] = 99

# MAX_ROUNDS = 300
# OPTIMIZE_ROUNDS = False
# LEARNING_RATE = 0.07
# EARLY_STOPPING_ROUNDS = 50  


In [80]:
lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

n_est = 300  
xgb_model = xgb.XGBClassifier(n_estimators=n_est,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=.05, 
                        subsample=.9, 
                        colsample_bytree=.8,
                        gamma=1,
                        reg_alpha=0,
                        reg_lambda=1,
                        nthread=2)
# xgb_model = XGBClassifier(    
#                         n_estimators=MAX_ROUNDS,
#                         max_depth=4,
#                         objective="binary:logistic",
#                         learning_rate=LEARNING_RATE, 
#                         subsample=.8,
#                         min_child_weight=6,
#                         colsample_bytree=.8,
#                         scale_pos_weight=1.6,
#                         gamma=10,
#                         reg_alpha=8,
#                         reg_lambda=1.3,
#                      )
# nn_model = sklearn.neural_network.MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#        beta_2=0.999, early_stopping=False, epsilon=1e-08,
#        hidden_layer_sizes=(50, 50, 50), learning_rate='constant',
#        learning_rate_init=0.001, max_iter=200, momentum=0.9,
#        nesterovs_momentum=True, power_t=0.5, random_state=None,
#        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
#        verbose=False, warm_start=False)

In [81]:
#log_model = LogisticRegression(penalty = 'l2', C = 0.95)
log_model = LogisticRegression()

        
stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (xgb_model, lgb_model, lgb_model2, lgb_model3))       

In [82]:
y_pred = stack.fit_predict(X,y, test)  

Fit XGBClassifier fold 1
Fit XGBClassifier fold 2
Fit XGBClassifier fold 3


/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)


Fit LGBMClassifier fold 1


/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)


Fit LGBMClassifier fold 2


...

Fit LGBMClassifier fold 1


/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)


Fit LGBMClassifier fold 2


/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/yucongweng/anaconda3/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)


Fit LGBMClassifier fold 3
Stacker score: 0.64258


Stacker score: 0.64255

In [83]:
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_pred
sub.to_csv('stacked_1.csv', index=False)

In [84]:
xgb_cv = pd.read_csv('xgb_submit.csv')

In [94]:
xgb_cv.head()

,id,target
0,0,0.039913
1,1,0.041650
2,2,0.037540
3,3,0.022209
4,4,0.054698


In [129]:
xgb_cv.target.values

array([ 0.039913,  0.04165 ,  0.03754 , ...,  0.056282,  0.036531,
        0.047749])

In [85]:
stack_sub = 2/(1/xgb_cv.target.values+1/y_pred)

In [127]:
stack_sub = (xgb_cv.target+y_pred)/2

In [87]:
sub2 = pd.DataFrame()
sub2['id'] = id_test
sub2['target'] = stack_sub
sub2.to_csv('stacked_2.csv', index=False)